# 阶段一: 候选短语的自动化发现 (The Discovery Engine)

**总目标:** 从约17万条中国新闻中，通过三种互补的算法（Gensim Phrases, spaCy NER, spaCy Noun Chunks），大规模挖掘潜在的、有意义的多词短语，为后续的人工审核环节提供一个高质量的“候选池”。

---
### 步骤 1.0: 环境设置与库导入

In [7]:
# 导入必要的库
import pandas as pd
import spacy
import time
import re
from gensim.models.phrases import Phrases
from collections import Counter
from tqdm.auto import tqdm
import pickle # 用于保存中间结果

# --- 配置区 ---

ALIYUN_OSS_PATH = ''    #ALIYUN_OSS_PATH = '/mnt/aliyun_oss/'

# 输入文件: 筛选并去重后的中国相关文章数据 (CSV格式)
INPUT_CHINA_DATA_PATH = ALIYUN_OSS_PATH + "../data_processed/final_china_news.csv"

# 中间产出文件: 保存分词结果，避免重复运行耗时步骤
TOKEN_LISTS_PATH = ALIYUN_OSS_PATH + "data_processed/intermediate_raw_token_lists.pkl"

# 最终产出文件: 保存本阶段发现的所有候选短语，以便下一阶段整合
CANDIDATES_GENSIM_PATH = ALIYUN_OSS_PATH + "data_processed/candidates_gensim.pkl"
CANDIDATES_NER_PATH = ALIYUN_OSS_PATH + "data_processed/candidates_ner.pkl"
CANDIDATES_NOUN_CHUNKS_PATH = ALIYUN_OSS_PATH + "data_processed/candidates_noun_chunks.pkl"

print("--- 环境准备 ---")
print(f"输入文件路径: {INPUT_CHINA_DATA_PATH}")

# --- 加载spaCy模型 ---
# 这里我们需要完整的模型来进行NER和语法分析
print("\n正在加载spaCy模型 'en_core_web_md' (完整模式)...")
start_time = time.time()
try:
    nlp = spacy.load("en_core_web_sm")
    print(f"spaCy模型加载成功！耗时: {time.time() - start_time:.2f} 秒")
except OSError:
    print("错误: spaCy模型 'en_core_web_md' 未安装。")
    print("请在你的终端或命令行中运行: python -m spacy download en_core_web_md")


--- 环境准备 ---
输入文件路径: ../data_processed/final_china_news.csv

正在加载spaCy模型 'en_core_web_md' (完整模式)...
spaCy模型加载成功！耗时: 0.39 秒


### 步骤 1.1: 前置任务 - 文本预分词

**逻辑与目的:**
为了让后续的`Gensim Phrases`模型能够高效运行，我们需要先将长文本切分成单词列表。此处的处理非常轻量，目的是保留尽可能多的原始短语结构，包括大小写和特殊字符组合。

**动作:**
1. 加载数据。
2. 创建分词器。
3. 批量分词，只进行最基础的过滤（非标点、非空格），保留原始文本。
4. 生成并保存内存对象 `raw_token_lists`。

In [ ]:
print(f"\n--- 步骤 1.1: 文本预分词 ---")
print(f"正在从 {INPUT_CHINA_DATA_PATH} 读取数据...")

start_time = time.time()
# 指定dtype为str，避免pandas自动推断类型出错
df = pd.read_csv(INPUT_CHINA_DATA_PATH, dtype={'DATE': str, 'CONTENT': str})
# 清理可能存在的空内容行
df.dropna(subset=['CONTENT'], inplace=True)
load_time = time.time() - start_time
print(f"数据加载完成！耗时: {load_time:.2f} 秒。 数据形状: {df.shape}")

# --- 批量分词 ---
print("\n开始批量分词（保留原始大小写和结构），这将花费一些时间...")
# 预估时间：17万条新闻，约15 - 40分钟
start_time = time.time()

raw_token_lists = []
# 使用nlp.pipe进行高效批量处理
# 我们只对CONTENT列进行操作
for doc in tqdm(nlp.pipe(df['CONTENT'], batch_size=500), total=len(df), desc="预分词中"):
    # 过滤条件：只要token既不是纯标点符号，也不是纯空格，就保留其原始文本
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
    raw_token_lists.append(tokens)

tokenize_time = time.time() - start_time
print(f"预分词处理完成！耗时: {tokenize_time/60:.2f} 分钟。")
print(f"已生成 {len(raw_token_lists)} 篇文章的原始token列表。")
print("\n示例 (第一篇文章的前20个token):")
print(raw_token_lists[0][:20])

# --- 保存中间结果 ---
print(f"\n正在将预分词结果保存到 {TOKEN_LISTS_PATH}...")
with open(TOKEN_LISTS_PATH, 'wb') as f:
    pickle.dump(raw_token_lists, f)
print("保存成功！")


--- 步骤 1.1: 文本预分词 ---
正在从 ../data_processed/final_china_news.csv 读取数据...
数据加载完成！耗时: 5.88 秒。 数据形状: (178273, 2)

开始批量分词（保留原始大小写和结构），这将花费一些时间...


预分词中:   0%|          | 0/178273 [00:02<?, ?it/s]


KeyboardInterrupt: 

### 步骤 1.2: 候选发现策略一 - 统计共现 (Gensim `Phrases`)

**逻辑与目的:**
利用统计学原理，找出那些“粘合度”很高的、频繁在一起出现的单词组合。此方法对发现**专业术语**和高频实体特别有效。

**动作:**
1.  标准化输入：创建`raw_token_lists`的小写副本。
2.  训练`Phrases`模型。
3.  导出候选短语及其得分。

In [ ]:
print(f"\n--- 步骤 1.2: Gensim Phrases 发现 ---")

# --- 1. 标准化输入 ---
print("正在为Gensim Phrases准备小写化的token列表...")
# 创建一个深拷贝，避免修改原始的raw_token_lists
documents_for_gensim = [[token.lower() for token in doc] for doc in tqdm(raw_token_lists, desc="转小写")]

# --- 2. 训练模型 ---
print("\n开始训练Phrases模型...")
# 预估时间：17万篇文章，约1 - 10分钟
start_time = time.time()
# min_count: 一个词组至少要在语料中出现20次
# threshold: 粘合度阈值，10是一个常用的起点
phrases_model = Phrases(documents_for_gensim, min_count=20, threshold=10.0, delimiter=b'_')
train_time = time.time() - start_time
print(f"Phrases模型训练完成！耗时: {train_time:.2f} 秒。")

# --- 3. 导出候选 ---
print("\n正在导出Gensim发现的候选短语...")
# phrases_model.export_phrases() 返回一个迭代器，我们将其转换为字典
gensim_candidates_raw = phrases_model.export_phrases(documents_for_gensim)
# 将bytes转换为utf-8字符串，并存入字典
gensim_candidates = {phrase.decode('utf-8'): score for phrase, score in gensim_candidates_raw}
print(f"Gensim Phrases发现了 {len(gensim_candidates)} 个候选短语。")

# --- 保存结果 ---
print(f"\n正在将Gensim候选结果保存到 {CANDIDATES_GENSIM_PATH}...")
with open(CANDIDATES_GENSIM_PATH, 'wb') as f:
    pickle.dump(gensim_candidates, f)
print("保存成功！")

### 步骤 1.3: 候选发现策略二 - 命名实体识别 (spaCy NER)

**逻辑与目的:**
利用预训练的深度学习模型，直接、精准地识别出文本中的人名、地名、组织机构名等**命名实体**。

**动作:**
1.  批量识别原始文本。
2.  提取我们感兴趣的实体类型。
3.  计数并收集。

In [ ]:
print(f"\n--- 步骤 1.3: spaCy NER 发现 ---")
print("开始批量提取命名实体，这将是本阶段最耗时的步骤之一...")
# 预估时间：17万篇文章，约 0.75 - 2 小时
start_time = time.time()

ner_candidates = Counter()
# 定义我们感兴趣的实体类型
TARGET_ENTITY_LABELS = {'ORG', 'PERSON', 'GPE', 'NORP', 'FAC', 'LOC', 'PRODUCT', 'EVENT'}

# 再次使用高效的nlp.pipe，在原始CONTENT列上操作
for doc in tqdm(nlp.pipe(df['CONTENT'], batch_size=200), total=len(df), desc="NER实体提取中"):
    for ent in doc.ents:
        # 检查实体标签是否在我们感兴趣的范围内
        if ent.label_ in TARGET_ENTITY_LABELS:
            # 清理实体文本，移除首尾空格
            entity_text = ent.text.strip()
            # 我们只对包含空格的多词实体感兴趣
            if ' ' in entity_text:
                # 转为小写后计入Counter，以实现聚合
                ner_candidates[entity_text.lower()] += 1

ner_time = time.time() - start_time
print(f"命名实体提取完成！耗时: {ner_time/60:.2f} 分钟。")
print(f"spaCy NER提取了 {len(ner_candidates)} 个唯一的多词实体候选。")

# --- 保存结果 ---
print(f"\n正在将NER候选结果保存到 {CANDIDATES_NER_PATH}...")
with open(CANDIDATES_NER_PATH, 'wb') as f:
    pickle.dump(ner_candidates, f)
print("保存成功！")

### 步骤 1.4: 候选发现策略三 - 语法结构分析 (spaCy `noun_chunks`)

**逻辑与目的:**
作为补充，利用语法分析器找出所有符合“名词短语”结构的部分，捕获非标准实体和描述性短语。

**动作:**
1.  批量分析原始文本。
2.  提取名词短语。
3.  清洗并计数。

In [ ]:
print(f"\n--- 步骤 1.4: spaCy Noun Chunks 发现 ---")
print("开始批量提取名词短语，这也将花费较长时间...")
# 预估时间：17万篇文章，约 0.75 - 2 小时
start_time = time.time()

noun_chunk_candidates = Counter()
# 定义一个简单的限定词列表，用于清理
determiners = {'the', 'a', 'an', 'my', 'your', 'his', 'her', 'its', 'our', 'their'}

# 同样在原始CONTENT列上操作
for doc in tqdm(nlp.pipe(df['CONTENT'], batch_size=200), total=len(df), desc="名词短语提取中"):
    for chunk in doc.noun_chunks:
        # 1. 获取名词短语文本并转为小写
        phrase_text = chunk.text.lower()

        # 2. 清理首尾的冠词/限定词
        words = phrase_text.split()
        if words:
            if words[0] in determiners:
                words = words[1:]
            if words and words[-1] in determiners:
                words = words[:-1]

        cleaned_phrase = " ".join(words).strip()

        # 3. 过滤掉不合格的短语
        if cleaned_phrase and ' ' in cleaned_phrase: # 确保是多词短语
            noun_chunk_candidates[cleaned_phrase] += 1

noun_chunk_time = time.time() - start_time
print(f"名词短语提取完成！耗时: {noun_chunk_time/60:.2f} 分钟。")
print(f"spaCy提取了 {len(noun_chunk_candidates)} 个唯一名词短语候选。")

# --- 保存结果 ---
print(f"\n正在将名词短语候选结果保存到 {CANDIDATES_NOUN_CHUNKS_PATH}...")
with open(CANDIDATES_NOUN_CHUNKS_PATH, 'wb') as f:
    pickle.dump(noun_chunk_candidates, f)
print("保存成功！")

print("\n\n=== 阶段一全部完成！ ===")
print("所有三种来源的候选短语都已生成并保存。下一步是运行 '05_Candidate_Integration_and_Review.ipynb' 来整合和审核这些结果。")